### Figure 6 | Circulating immune-cell composition remains largely unchanged following vaccination
Inferred PBMC subset proportions across timepoints indicate minimal compositional shifts despite transcriptional remodeling.